# Imports and Constants

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os

import decoding
from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *

C:\Users\nicol\miniconda3\lib\site-packages\nilearn\datasets\__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length = nb_runs * len(classes)
subjects_ids = range(1, 24)  # TODO modify this to get all subjects (here we only have 1 for the example)
n_subjects = len(subjects_ids)
n_perms = 1000
class_labels = [""] * (2 * length)
for i in range(length * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length])
labels["aud"] = np.array(class_labels[length:])
labels_same = labels["vis"]

classical_tasks_regions = [(["vis"], ["V5_R", "V5_L"]),
                           (["aud"], ["PT_R", "PT_L"]),
                           (["aud"], ["V5_R", "V5_L"])]

cross_modal_task_regions = [(["vis", "aud"], ["V5_R", "V5_L"])]

scaler = sklearn.preprocessing.StandardScaler()
classifier = sklearn.svm.SVC(C = 1, kernel = 'linear', random_state=SEED)  # other parameters are default parameters
pipeline = Pipeline([('std', scaler), ('svm', classifier)])
decoder = Decoder(n_perm=n_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)

# Loading data

In [3]:
scores_perms = [dict() for _ in subjects_ids]
maps_masked = [dict() for _ in subjects_ids]

In [4]:
for i, subj_id in enumerate(subjects_ids):
    t_maps, beta_maps = get_maps([subj_id])
    masks = get_masks([subj_id], plot=False)
    maps = apply_mask_to_maps(t_maps, masks)
    maps_masked[i]["vis"] = get_part_of_maps(maps, 0, length)  # maps acquired for the vision experiment
    maps_masked[i]["aud"] = get_part_of_maps(maps, length, 2 * length)  # maps acquired for the audition experiment
    del t_maps ; del beta_maps ; del masks  # to relieve memory
    print("Loading done for subject "+str(subj_id)+"/"+str(n_subjects))

Loading done for subject 1/23
Loading done for subject 2/23
Loading done for subject 3/23
Loading done for subject 4/23
Loading done for subject 5/23
Loading done for subject 6/23
Loading done for subject 7/23
Loading done for subject 8/23
Loading done for subject 9/23
Loading done for subject 10/23
Loading done for subject 11/23
Loading done for subject 12/23
Loading done for subject 13/23
Loading done for subject 14/23
Loading done for subject 15/23
Loading done for subject 16/23
Loading done for subject 17/23
Loading done for subject 18/23
Loading done for subject 19/23
Loading done for subject 20/23
Loading done for subject 21/23
Loading done for subject 22/23
Loading done for subject 23/23


In [5]:
def average_dicos(dicos_):
    dico = dict()
    for key in dicos_[0]:
        dico[key] = np.mean([dic[key] for dic in dicos_])

    return dico

# Within-modality decoding

In [ ]:
cv_scores = [dict() for _ in subjects_ids]
p_values = [dict() for _ in subjects_ids]
for i, subj_id in enumerate(subjects_ids):
    # within-modality decoding : training on a task and decoding on other samples from same task
    for tasks, regions in classical_tasks_regions:
        cv_sc, p_val, scores_perm = decoder.classify_tasks_regions(maps_masked[i], labels, tasks, regions)
        cv_scores[i].update(cv_sc)
        p_values[i].update(p_val)
        scores_perms[i].update(scores_perm)
    print("Within-modality decoding done for subject "+str(subj_id)+"/"+str(n_subjects))

In [ ]:
# Compute groups results
within_modality_group_results = average_dicos(cv_scores)

# Cross-modal decoding

In [ ]:
cv_scores = [dict() for _ in subjects_ids]
p_values = [dict() for _ in subjects_ids]
for i, subj_id in enumerate(subjects_ids):
    # cross-modal decoding : training on a task and decoding on samples from another task
    for tasks, regions in cross_modal_task_regions:
        scores_cross_mod = decoder.cross_modal_decoding(maps_masked[i], labels, tasks, regions)
        cv_scores[i].update(scores_cross_mod)

    print("Cross-modal decoding done for subject "+str(subj_id)+"/"+str(n_subjects))

In [ ]:
# Compute groups results
cross_modality_group_results = average_dicos(cv_scores)

# Bootstrapping to assess group-level significance

In [6]:
import time

start_time = time.time()
n_bootstrap = 100_000
n_single_perm = 100

# 1. produce the 100 permuted datasets for each subject, and compute scores
scores_100_Perm = [None]*n_subjects
for i in range(n_subjects):
    labels_shuffled = decoder.produce_permuted_labels(labels_same, n_single_perm) # repeating for each subject, such that we don't obtain same permutations
    scores_dicts = [dict() for _ in range(n_single_perm)]
    for j in range(n_single_perm) :
        labels_dico = {"vis":labels_shuffled[j],"aud":labels_shuffled[j]}
        for tasks, regions in classical_tasks_regions :
            cv_sc,_,_ = decoder.classify_tasks_regions(maps_masked[i], labels_dico, tasks, regions, do_pval=False)
            scores_dicts[j].update(cv_sc)

    scores_100_Perm[i] = scores_dicts
    print("subject "+str(i)+" done")

intermed_time = time.time()-start_time
print("Running models done in "+str(intermed_time)+" seconds")

# 2. select randomly 100 000 times a score from each subject
scores_bootstrap = [None]*n_bootstrap
for i in range(n_bootstrap) :
    dicos = [dict() for _ in range(n_subjects)]
    for j in range(n_subjects) :
        dicos[j] = scores_100_Perm[j][random.randint(0,n_single_perm-1)]
    scores_bootstrap[i] = average_dicos(dicos)
    if i%10_000 == 0 :
        print("iteration n° "+str(i))

end_time = time.time() - intermed_time
print("Creating distribution done in "+str(end_time)+" seconds")

# 3. compare with our obtained result


subject 0 done
subject 1 done
subject 2 done
subject 3 done
subject 4 done
subject 5 done
subject 6 done
subject 7 done
subject 8 done
subject 9 done
subject 10 done
subject 11 done
subject 12 done
subject 13 done
subject 14 done
subject 15 done
subject 16 done
subject 17 done
subject 18 done
subject 19 done
subject 20 done
subject 21 done
subject 22 done
Running models done in 308.77601075172424 seconds
Creating distribution done in 1640972830.386041 seconds


# Saving results

In [9]:
create_directory("out")
save_dicts("cv_scores.csv", cv_scores, list(cv_scores[0].keys()), subjects_ids)
save_dicts("p_values.csv", p_values, list(p_values[0].keys()), subjects_ids)
save_dicts_perms("scores_perms.csv", scores_perms, subjects_ids)

In [7]:
save_dicts("bootstraps.csv", scores_bootstrap, list(scores_bootstrap[0].keys()),range(n_bootstrap))

# Plotting results (from files of saved results)

In [3]:
plt_directory = "plots"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)
plotter.plot_cv_pval("out/cv_scores.csv", "cv score", plotter.cv_scores_dir, chance_level = True)
plotter.plot_cv_pval("out/p_values.csv", "p-value", plotter.p_values_dir)
plotter.plot_perms_scores("out/scores_perms.csv", n_perms)

# Visualizing differences between audition and vision in the voxels of a ROI

In [ ]:
region = "V5_R"
colors = ["red","tomato","coral","orange","deepskyblue","cyan","blue","royalblue"]
n_voxels = maps_masked[0]["vis"][0][region].shape[1]
mean_aud = dict()
mean_vis = dict()

for cla in classes :
    mean_aud[cla] = np.zeros(n_voxels)
    mean_vis[cla] = np.zeros(n_voxels)


for i in range(n_subjects):
    for j, cla in enumerate(classes) :
        mean_vis[cla] += np.mean(maps_masked[i]["vis"][0][region][j*12:(j+1)*12],axis=0)/n_subjects
        mean_aud[cla] += np.mean(maps_masked[i]["aud"][0][region][j*12:(j+1)*12],axis=0)/n_subjects

plt.rcParams.update({'font.size': 15})
plt.figure(figsize=(12,8))

idx = 0
for cla in classes :
    plt.plot(range(n_voxels), mean_vis[cla], label = "vis - "+cla, color = colors[idx])
    idx += 1

for cla in classes :
    plt.plot(range(n_voxels), mean_aud[cla], label = "aud - "+cla, color = colors[idx])
    idx += 1

plt.xlabel("voxel id")
plt.ylabel("intensity")
plt.title("Average voxel intensities for right V5")
plt.legend()
plt.savefig("plots/why_normalize_cross_modal.png")